# scrapy中spiders用法

Spider类定义了如何爬去某个网站，包括爬取的动作以及如何从网页内容中提取结构化的数据，总的来说spider就是定义爬取的动作以及分析某个网页

## 工作流程分析

1. 以初始的URL初始化Request，并设置回调函数，当该request下载完毕并返回时，将生成response，并作为参数传给回调函数. spider中初始的request是通过start_requests()来获取的。start_requests()获取 start_urls中的URL，并以parse以回调函数生成下面的Requests.
2. 在回调函数内分析返回的网页内容，可以返回Item对象，或者Dict，或者Request，以及是一个包含三者的可迭代的容器，返回的Request对象之后会经过Scrapy处理，下载相应的内容，并调用设置的callback函数
3. 在回调函数内，可以通过lxml，bs4，xpath,css等方法获取我们想要的内容生成item
4. 最后将item传递给Pipeline处理

通过简单的分析源码来理解:
通常在写spiders下写爬虫的时候，并没有写`start_requests`来处理`start_urls`中的url，这是因为在继承的scrapy.Spider中已经写过了，可以点开scrapy.Spider查看分析, 如下图:

![](images/12.png)

通过上述代码可以看到在父类里实现了`start_requests`方法，通过`make_requests_from_url`做了Request请求

如下图所示的一个例子，parse回调函数中的response就是父类中`start_requests`方法调用`make_requests_from_url`返回的结果，并且在parse回调函数中我们可以继续返回Request,如下属代码中`yield Request()`

![](images/13.png)

## spider常用属性

我们所有自己写的爬虫都是继承自`spider.Spider`这个类

* name : 定义爬虫名字，通过命令启动的时候用的就是这个名字，这个名字必须是唯一的
* allowed_domains: 包含了spider允许爬取的域名列表。当`offsiteMiddleware`启用时，域名不在列表中URL不会被访问,所以在爬虫文件中，每次生成Request请求时都会进行和这里的域名进行判断
* start_urls: 起始的url列表. 这里会通过spider.Spider方法中会调用start_request循环请求这个列表中每个地址。
* custom_settings: 自定义配置，可以覆盖settings的配置，主要用于对爬虫特定需求设置, 设置是以字典方式:`custom_settings = {}`
* from_crawler: 这是一个类方法，定义这样一个类方法，可以通过`crawler.settings.get()`这种方式获取settings配置文件中的信息，同时这个也可以在pipeline中使用
* start_requests(): 这个方法必须返回一个可迭代对象，该对象包含了spider用于爬取的第一个Request请求,这个方法是在被继承的父类中spider.Spider中写的，默认是通过get请求，如果我们需要修改最开始的这个请求，可以重写这个方法，如我们想通过post请求
* make_requests_from_url(url): 这个也是在父类中start_requests调用的，当然这个方法我们也可以重写
* parse(response): 这个是默认的回调函数, 负责处理response并返回处理的数据以及跟进的url, 该方法以及其他的Request回调函数必须返回一个包含Request或Item的可迭代对象